In [385]:
import pandas as pd; import numpy as np; import math
excel_file=pd.read_excel('C:/ANS/test.xlsx'); excel_file=pd.DataFrame(excel_file)                               # 파일 불러온 후 데이터프레임화
excel_value=np.array(excel_file.values)                                                                         # value만 추출
flatten_excel_value=list(set(list(excel_value.flatten())))                                                      # value 1차원화, nan 제외 중복값 삭제
only_str=[x for x in flatten_excel_value if type(x) !=float]; only_str=[x for x in only_str if type(x) !=int]   # 제거 성분(row)를 위해 int,float값 삭제
only_str=[str(x) for x in only_str]                                                                             # 혹시몰라 한 번더 문자화
row=[x for x in only_str if x !='nan']; col=excel_file.columns[excel_file.columns.str.contains('Class')]        # row정의 nan값 삭제, 제거 성분만 남김 | col정의 'Class'가 들어간 열만 뽑아 객체만 남김
n=len(row); m=len(col)                                                                                          # 새롭게 만들 행렬의 행x열 크기를 위한 n,m설정
count_data=excel_file[excel_file.columns[excel_file.columns.str.contains('Count')]]                             # 'Count'가 들어간 열만 뽑아 카운트 데이터 생성 ## 'Class'와 'Count'는 추후에 함수화시 변수로 만들어 사용할 예정
count_data=count_data.rename(columns={'Count':'Count.0'})                                                       # 첫 Column name이 Count여서 편의성과 통일성을 위해 교정
count_data=count_data.fillna(0)                                                                                 # nan값 0으로 교체
removal_ingredient=excel_file[col]                                                                              # 각 Class의 제거되는 요소 
commit_first_value=np.zeros((n,m))                                                                              # 최종 데이터프레임의 value값. 최종 형태도 카운트 데이터이기에 np.zeros사용
commit=pd.DataFrame(commit_first_value,row,col)                                                                 # 데이터 프레임화                                                                   
for i in range(len(col)):                                                                                       # Class 범위
    removal_ingredient_list=list(removal_ingredient.values[:,i])                                                # 각 Class의 제거되는 요소 추출
    for j in range(len(removal_ingredient_list)):                                                               # 추출된 요소 범위
        if removal_ingredient_list[j] in row:                                                                   # 추출된 요소가 row에 포함되어 있는지 여부 판별
            commit[commit.columns[i]][removal_ingredient_list[j]]=count_data['Count.{}'.format(i)][j]           # 포함되어 있을때 위치에 알맞는 카운트 데이터가 새 행렬 commit에 들어감
        else: pass
commit


,Class_GCF_000016645.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000243275.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000246945.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000378485.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000419685.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000422685.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000422705.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000422725.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000425425.lenfil.cds.fas_bacmet.table_class_cout_,Class_GCF_000425445.lenfil.cds.fas_bacmet.table_class_cout_,...,Class_N2155-p2.gbk.gex4.cds.fas_bacmet.table_class_cout_,Class_N2270.gbk.cds.fas_bacmet.table_class_cout_,Class_N2469.gbk.cds.fas_bacmet.table_class_cout_,Class_N2550.gbk.cds.fas_bacmet.table_class_cout_,Class_N2820.gbk.cds.fas_bacmet.table_class_cout_,Class_N3904.gbk.cds.fas_bacmet.table_class_cout_,Class_N501239.gbk.cds.fas_bacmet.table_class_cout_,Class_N502536.gbk.cds.fas_bacmet.table_class_cout_,Class_N502540.gbk.cds.fas_bacmet.table_class_cout_,Class_N503310.gbk.cds.fas_bacmet.table_class_cout_
Acid,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Diamindine,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Biguanides,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Xanthene,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Organo-sulfate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Ni, Co",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Copper (Cu),0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acridine,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Phenanthridine,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arsenic (As),1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
